In [13]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

# from scipy.stats import binned_statistic_2d
# from scipy.ndimage import gaussian_filter
from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.cm as cm

# from abacusnbody.analysis.tsc import tsc_parallel
import time

from astropy.cosmology import FlatLambdaCDM, Planck18
import astropy.units as u

# Import packages

sys.path.append('../src/')
# from filter_utils import *
from SZstacker import SZMapStacker # type: ignore
from stacker import SimulationStacker
from loadIO import load_data    
from utils import fft_smoothed_map, gaussian_smoothed_map

# sys.path.append('../../illustrisPython/')
import illustris_python as il # type: ignore

import yaml
import argparse
from pathlib import Path
import glob

In [6]:
stacker = SZMapStacker('TNG100-1', 67, z=0.5,)

In [7]:
folderPath = stacker.snapPath(stacker.simType, pathOnly=True)
snaps = glob.glob(folderPath + 'snap_*.hdf5')
snap = snaps[0]

In [8]:
header = stacker.header

In [9]:
header

{'BoxSize': 75000.0,
 'Composition_vector_length': 0,
 'Flag_Cooling': 1,
 'Flag_DoublePrecision': 0,
 'Flag_Feedback': 1,
 'Flag_Metals': 0,
 'Flag_Sfr': 1,
 'Flag_StellarAge': 0,
 'Git_commit': b'd203ec8b07c7e2bdda5f608aa0babea46d603699',
 'Git_date': b'Thu Apr 7 14:14:27 2016 +0200',
 'HubbleParam': 0.6774,
 'MassTable': array([0.00000000e+00, 5.05574296e-04, 0.00000000e+00, 4.71972931e-05,
        0.00000000e+00, 0.00000000e+00]),
 'NumFilesPerSnapshot': 448,
 'NumPart_ThisFile': array([12312098, 13286645,        0, 27608866,  1054239,       95],
       dtype=int32),
 'NumPart_Total': array([1237981302, 1733600704,          0, 3467201408,  367811595,
             37067], dtype=uint32),
 'NumPart_Total_HighWord': array([1, 1, 0, 2, 0, 0], dtype=uint32),
 'Omega0': 0.3089,
 'OmegaBaryon': 0.0486,
 'OmegaLambda': 0.6911,
 'Redshift': 0.5030475232448832,
 'Time': 0.6653149581332802,
 'UnitLength_in_cm': 3.085678e+21,
 'UnitMass_in_g': 1.989e+43,
 'UnitVelocity_in_cm_per_s': 100000.0}

In [ ]:
def comoving_to_arcmin(L_com_kpch, z, cosmo=Planck18):
    """
    Convert a comoving length at redshift z into angular size [arcmin].
    
    Parameters
    ----------
    L_com_kpch : float
        Comoving length in kpc/h.
    z : float
        Redshift
    
    Returns
    -------
    theta_arcmin : float
        Angular size in arcminutes.
    """
    # Convert kpc/h -> Mpc
    L_com_Mpc = L_com_kpch / (1000.0 * cosmo.h)
    
    # Comoving distance to redshift z [Mpc]
    chi = cosmo.comoving_distance(z).value
    
    # Angular size in radians
    theta_rad = L_com_Mpc / chi
    
    # Convert to arcminutes
    return theta_rad * (180.0 / np.pi) * 60.0

print(comoving_to_arcmin(header['BoxSize'], 0.5))

195.54834571760415


In [24]:
cosmo.h

0.6766

In [21]:
cosmo = Planck18
z = 0.5
cosmo.comoving_distance(z)

<Quantity 1946.41663479 Mpc>

In [23]:
header['HubbleParam']

0.6774

In [22]:
def current_code(boxsize, z, cosmo=Planck18):
    dA = cosmo.angular_diameter_distance(z).to(u.kpc).value
    dA *= header['HubbleParam']  # Convert to kpc/h
    
    # Get the box size in angular units.
    theta_arcmin = np.degrees(boxsize / dA) * 60  # Convert to arcminutes # TODO: this is wrong I think!!
    print(f"Map size at z={z}: {theta_arcmin:.2f} arcmin")

current_code(header['BoxSize'], 0.5)

Map size at z=0.5: 293.32 arcmin
